# 1. Exploration.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import mytools as mt
import spacy

### 1.1 EXTRACTION DES QUESTIONS DE STACKOVERFLOW

### 1.2 CHARGEMENT DES QUESTIONS

In [2]:
data = pd.read_csv('data_stackoverflow.csv')
data.head()

,Id,Title,Body,Tags,Score,ViewCount,AnswerCount
0,11227809,Why is processing a sorted array faster than p...,"<p>In this C++ code, sorting the data (<em>bef...",<java><c++><performance><cpu-architecture><bra...,27241,1879023,25
1,2003505,How do I delete a Git branch locally and remot...,<p>Failed Attempts to Delete a Remote Branch:<...,<git><version-control><git-branch><git-push><g...,20386,11469758,41
2,1642028,What is the '-->' operator in C/C++?,"<p>After reading <a href=""http://groups.google...",<c++><c><operators><code-formatting><standards...,10153,1009735,26
3,1125968,"How do I force ""git pull"" to overwrite local f...",<p>How do I force an overwrite of local files ...,<git><version-control><overwrite><git-pull><gi...,9642,8493508,53
4,79923,What and where are the stack and heap?,<ul>\n<li>What are the stack and heap?</li>\n<...,<data-structures><memory-management><heap-memo...,9416,1930346,31


In [3]:
data.shape

(50000, 7)

In [4]:
data.set_index('Id', inplace=True)
data.head()

,Title,Body,Tags,Score,ViewCount,AnswerCount
Id,,,,,,
11227809,Why is processing a sorted array faster than p...,"<p>In this C++ code, sorting the data (<em>bef...",<java><c++><performance><cpu-architecture><bra...,27241,1879023,25
2003505,How do I delete a Git branch locally and remot...,<p>Failed Attempts to Delete a Remote Branch:<...,<git><version-control><git-branch><git-push><g...,20386,11469758,41
1642028,What is the '-->' operator in C/C++?,"<p>After reading <a href=""http://groups.google...",<c++><c><operators><code-formatting><standards...,10153,1009735,26
1125968,"How do I force ""git pull"" to overwrite local f...",<p>How do I force an overwrite of local files ...,<git><version-control><overwrite><git-pull><gi...,9642,8493508,53
79923,What and where are the stack and heap?,<ul>\n<li>What are the stack and heap?</li>\n<...,<data-structures><memory-management><heap-memo...,9416,1930346,31


In [5]:
data.shape

(50000, 6)

In [6]:
mt.stats(data)

,count,mean,std,min,25%,50%,75%,max
Score,50000.0,82.360,281.832,19.0,25.00,36.0,65.00,27241.0
ViewCount,50000.0,89390.496,221236.392,207.0,17163.75,38143.5,84763.75,11469758.0
AnswerCount,50000.0,6.055,5.841,1.0,3.00,4.0,7.00,141.0


In [7]:
mt.valeurs_manquantes(data)

,effectif,taux


### 1.3 SÉLECTION DES 10000 QUESTIONS LES PLUS VUES

In [8]:
data = data.sort_values(by='ViewCount', ascending=False)
data.head()

,Title,Body,Tags,Score,ViewCount,AnswerCount
Id,,,,,,
2003505,How do I delete a Git branch locally and remot...,<p>Failed Attempts to Delete a Remote Branch:<...,<git><version-control><git-branch><git-push><g...,20386,11469758,41
16956810,Find all files containing a specific text (str...,<p>How do I find all files containing a specif...,<linux><text><grep><directory><find>,7489,10975347,57
1125968,"How do I force ""git pull"" to overwrite local f...",<p>How do I force an overwrite of local files ...,<git><version-control><overwrite><git-pull><gi...,9642,8493508,53
4366730,How do I check if a string contains a specific...,<p>Consider:</p>\n\n<pre><code>$a = 'How are y...,<php><string><substring><contains><string-matc...,2657,6700920,36
11346283,Renaming column names in Pandas,<p>I want to change the column labels of a Pan...,<python><pandas><replace><dataframe><rename>,2967,6433881,33


In [9]:
data = data.iloc[:10000,:]
data.shape

(10000, 6)

In [10]:
mt.stats(data)

,count,mean,std,min,25%,50%,75%,max
Score,10000.0,243.171,599.654,19.0,61.00,114.0,223.0,27241.0
ViewCount,10000.0,307159.009,427459.648,104635.0,134295.75,184857.5,311065.0,11469758.0
AnswerCount,10000.0,11.192,9.338,1.0,5.00,8.0,14.0,141.0


### 1.4 PRÉTRAITEMENT DES FEATURES TEXTUELLES

In [11]:
import re

In [12]:
# Charger un modèle linguistique anglais
nlp = spacy.load("en_core_web_lg")

In [17]:
# Fonction pour traiter les données textuelles.
def process_text(text, allowed_words=None):

    # Supprimer les caractères qui ne sont pas des lettres :
    text = re.sub(r'[^a-zA-Z\s]', ' ', text)
    
    doc = nlp(text)
    
    # Standardisation des mots et exclusion des stopwords :
    if allowed_words:
        words = [token.lemma_.lower() for token in doc if not token.is_stop and token.lemma_.lower() in allowed_words]
    else:
        words = [token.lemma_.lower() for token in doc if not token.is_stop]
    
    return words

In [18]:
data.Tags = data.Tags.apply(process_text)

In [19]:
data.head()

,Title,Body,Tags,Score,ViewCount,AnswerCount
Id,,,,,,
2003505,How do I delete a Git branch locally and remot...,<p>Failed Attempts to Delete a Remote Branch:<...,"[ , git, , version, control, , git, branch, ...",20386,11469758,41
16956810,Find all files containing a specific text (str...,<p>How do I find all files containing a specif...,"[ , linux, , text, , grep, , directory, , ...",7489,10975347,57
1125968,"How do I force ""git pull"" to overwrite local f...",<p>How do I force an overwrite of local files ...,"[ , git, , version, control, , overwrite, ,...",9642,8493508,53
4366730,How do I check if a string contains a specific...,<p>Consider:</p>\n\n<pre><code>$a = 'How are y...,"[ , php, , string, , substring, , contain, ...",2657,6700920,36
11346283,Renaming column names in Pandas,<p>I want to change the column labels of a Pan...,"[ , python, , panda, , replace, , dataframe...",2967,6433881,33


In [22]:
Tags = data.Tags.explode().unique().tolist()
len(Tags)

4515

In [23]:
data.Title = data.Title.apply(lambda x: process_text(x, allowed_words=Tags))
data.Body = data.Body.apply(lambda x: process_text(x, allowed_words=Tags))

In [24]:
data.head()

,Title,Body,Tags,Score,ViewCount,AnswerCount
Id,,,,,,
2003505,"[delete, git, branch]","[ , p, fail, delete, remote, branch, , p, pr...","[ , git, , version, control, , git, branch, ...",20386,11469758,41
16956810,"[find, file, contain, text, , string, , linux]","[ , p, find, file, contain, string, text, file...","[ , linux, , text, , grep, , directory, , ...",7489,10975347,57
1125968,"[force, , git, pull, , overwrite, local, file]","[ , p, force, overwrite, local, file, , code,...","[ , git, , version, control, , overwrite, ,...",9642,8493508,53
4366730,"[check, string, contain, word]","[ , p, , p, pre, , code, , , , contai...","[ , php, , string, , substring, , contain, ...",2657,6700920,36
11346283,"[rename, column, name, panda]","[ , p, change, column, label, pandas, datafram...","[ , python, , panda, , replace, , dataframe...",2967,6433881,33


In [25]:
# Convertir les listes de chaînes de caractères en textes :
data.Title = data.Title.apply(lambda x: ' '.join(x))
data.Body = data.Body.apply(lambda x: ' '.join(x))
data.Tags = data.Tags.apply(lambda x: ' '.join(x))

In [26]:
data.head()

,Title,Body,Tags,Score,ViewCount,AnswerCount
Id,,,,,,
2003505,delete git branch,p fail delete remote branch p pre class ...,git version control git branch git pus...,20386,11469758,41
16956810,find file contain text string linux,p find file contain string text file content...,linux text grep directory find,7489,10975347,57
1125968,force git pull overwrite local file,p force overwrite local file code git pull...,git version control overwrite git pull...,9642,8493508,53
4366730,check string contain word,p p pre code contain echo ...,php string substring contain string ...,2657,6700920,36
11346283,rename column name panda,p change column label pandas dataframe p p...,python panda replace dataframe rename,2967,6433881,33


In [27]:
data.to_csv('questions_pretraitees.csv')